In [1]:
from temdb_client import create_client
from datetime import datetime

### Initialize the TEMdb Client

In [ ]:
client = create_client("http://localhost:80")

In [ ]:
import asyncio
import logging
from typing import List, Dict, Any

from temdb_client import create_client, AsyncTEMdbClient
from temdb_client.models.specimen import SpecimenCreate
from temdb_client.models.block import BlockCreate
from temdb_client.models.substrate import SubstrateCreate, Aperture
from temdb_client.models.cutting_session import CuttingSessionCreate
from temdb_client.models.section import SectionCreate
from temdb_client.models.roi import ROICreate
from temdb_client.models.task import AcquisitionTaskCreate

API_BASE_URL = "http://localhost:80"
API_KEY = None 
SPECIMEN_ID = "TESTSPEC001"
BLOCK_ID = "TESTBLOCK001"
TAPE1_ID = "TESTTAPE001"
TAPE2_ID = "TESTTAPE002"
SESSION_ID = "TESTCUT001"
NUM_APERTURES_PER_TAPE = 2500
TOTAL_SECTIONS = NUM_APERTURES_PER_TAPE * 2

def create_square_roi_params() -> Dict[str, Any]:
    size_nm = 1_000_000 # 1mm
    points = [
        [0.0, 0.0],
        [size_nm, 0.0],
        [size_nm, size_nm],
        [0.0, size_nm]
    ]
    return {"shape": "polygon", "polygon_nm": {"points": points}}

async def run_test_case():
    """Runs the test case to populate TEMdb."""

    print("Initializing TEMdb client...")
    client: AsyncTEMdbClient = create_client(
        base_url=API_BASE_URL,
        api_key=API_KEY,
        async_mode=True,
        debug=True
    )

    async with client:
        print("--- Step 1: Create Specimen ---")
        specimen_data = SpecimenCreate(
            specimen_id=SPECIMEN_ID,
            description=f"Test specimen for {TOTAL_SECTIONS} sections"
        )
        specimen = await client.specimen.create(specimen_data)
        print(f"Created Specimen: {specimen.specimen_id}")

        print("--- Step 2: Create Block ---")
        block_data = BlockCreate(
            block_id=BLOCK_ID,
            specimen_id=specimen.specimen_id,
            microCT_info={"scan_id": "CT_SCAN_XYZ"}
        )
        block = await client.block.create(block_data)
        print(f"Created Block: {block.block_id}")

        print("--- Step 3: Create Substrates (Tapes) ---")
        tape1_apertures = [Aperture(uid=f"{TAPE1_ID}_A{i}", index=i, status="available") for i in range(NUM_APERTURES_PER_TAPE)]
        tape2_apertures = [Aperture(uid=f"{TAPE2_ID}_A{i}", index=i, status="available") for i in range(NUM_APERTURES_PER_TAPE)]

        tape1_data = SubstrateCreate(
            media_id=TAPE1_ID,
            media_type="tape",
            apertures=tape1_apertures,
            status="new",
            uid=TAPE1_ID
        )
        tape2_data = SubstrateCreate(
            media_id=TAPE2_ID,
            media_type="tape",
            apertures=tape2_apertures,
            status="new",
            uid=TAPE2_ID
        )

        tape1 = await client.substrate.create(tape1_data)
        print(f"Created Substrate: {tape1.media_id}")
        tape2 = await client.substrate.create(tape2_data)
        print(f"Created Substrate: {tape2.media_id}")

        print("--- Step 4: Create Cutting Session ---")
        session_data = CuttingSessionCreate(
            cutting_session_id=SESSION_ID,
            block_id=block.block_id,
            start_time=datetime.datetime.now(datetime.timezone.utc),
            sectioning_device="VirtualMicrotome",
            media_type="tape",
            specimen_id=specimen.specimen_id 
        )
        session = await client.cutting_session.create(session_data)
        print(f"Created Cutting Session: {session.cutting_session_id}")

        print(f"--- Step 5: Create {TOTAL_SECTIONS} Sections (Loop) ---")

        all_sections = []
        for i in range(TOTAL_SECTIONS):
            section_num = i + 1
            if i < NUM_APERTURES_PER_TAPE:
                current_tape = tape1
                current_apertures = tape1_apertures
                aperture_idx_on_tape = i
            else:
                current_tape = tape2
                current_apertures = tape2_apertures
                aperture_idx_on_tape = i - NUM_APERTURES_PER_TAPE

            aperture = current_apertures[aperture_idx_on_tape]

            section_data = SectionCreate(
                cutting_session_id=session.cutting_session_id,
                media_id=current_tape.media_id,
                section_number=section_num,
                timestamp=datetime.datetime.now(datetime.timezone.utc), 
                aperture_index=aperture.index,
                aperture_uid=aperture.uid
            )
            try:
                created_section = await client.section.create(section_data)
                all_sections.append(created_section)
                if (i + 1) % 100 == 0: 
                     print(f"  Created section {i+1}/{TOTAL_SECTIONS}")
            except Exception as e:
                print(f"  Failed to create section {section_num}: {e}")
                raise e 
        print(f"Finished creating {len(all_sections)} sections.")

        print(f"--- Step 6: Create {len(all_sections)} ROIs (Loop) ---")
        all_rois = []
        roi_params = create_square_roi_params()
        for i, section in enumerate(all_sections):
            roi_id = section.section_number 
            roi_data = ROICreate(
                roi_id=roi_id,
                section_id=section.section_id,
                specimen_id=section.specimen_id, 
                block_id=section.block_id,
                section_number=section.section_number,
                roi_parameters=roi_params,
                parent_roi_id=None 
            )
            try:
                created_roi = await client.roi.create(roi_data)
                all_rois.append(created_roi)
                if (i + 1) % 100 == 0: 
                    print(f"  Created ROI {i+1}/{len(all_sections)}")
            except Exception as e:
                print(f"  Failed to create ROI for section {section.section_id}: {e}")
                raise e
        print(f"Finished creating {len(all_rois)} ROIs.")


        print(f"--- Step 7: Create {len(all_rois)} Acquisition Tasks (Batch) ---")
        tasks_to_create: List[AcquisitionTaskCreate] = []
        for roi in all_rois:
            task_id = f"TASK_{roi.specimen_id}_{roi.block_id}_{roi.roi_id}"
            task_data = AcquisitionTaskCreate(
                task_id=task_id,
                specimen_id=roi.specimen_id,
                block_id=roi.block_id,
                roi_id=roi.roi_id,
                tags=["bulk_test_case"]
                
            )
            tasks_to_create.append(task_data)

        if tasks_to_create:
            try:
                chunk_size = 500
                created_tasks_count = 0
                for j in range(0, len(tasks_to_create), chunk_size):
                    chunk = tasks_to_create[j:j + chunk_size]
                    print(f"  Submitting task batch {j//chunk_size + 1} ({len(chunk)} tasks)...")
                    batch_result = await client.acquisition_task.create_batch(chunk)
                    created_tasks_count += len(batch_result)
                    print(f"  ...batch created {len(batch_result)} tasks.")
                print(f"Finished creating {created_tasks_count} acquisition tasks via batch.")
            except Exception as e:
                 print(f"  Failed to create task batch: {e}")
        else:
             print("No ROIs available to create tasks for.")

        print("--- Test Case Script Finished ---")


if __name__ == "__main__":
    print("Running TEMdb Test Case Script...")
    logging.basicConfig(level=logging.INFO)
    logging.getLogger("temdb_client").setLevel(logging.DEBUG) 

    try:
        loop = asyncio.get_running_loop()
        print("Detected running event loop. Scheduling coroutine.")
        
        if loop.is_running():
            
             print("WARNING: Cannot reliably run in already running loop from __main__.")
             print("         Consider running 'await run_test_case()' directly in the cell.")
        else:
             loop.run_until_complete(run_test_case())

    except RuntimeError:
        print("No running event loop detected. Using asyncio.run().")
        asyncio.run(run_test_case())

    print("Script completed.")


In [ ]:
await run_test_case()

In [ ]:
import asyncio
import datetime
import logging
import math
from typing import List, Dict, Any, Optional, Literal


from temdb_client import create_client, AsyncTEMdbClient
from temdb_client.models.specimen import SpecimenCreate
from temdb_client.models.block import BlockCreate
from temdb_client.models.substrate import SubstrateCreate, Aperture
from temdb_client.models.cutting_session import CuttingSessionCreate
from temdb_client.models.section import SectionCreate
from temdb_client.models.roi import ROICreate
from temdb_client.models.task import AcquisitionTaskCreate

API_BASE_URL = "http://localhost:80" 
API_KEY = None 


def create_square_roi_params(size_nm: float = 1_000_000) -> Dict[str, Any]:
    """Generates parameters for a square ROI."""
    points = [
        [0.0, 0.0], [size_nm, 0.0], [size_nm, size_nm], [0.0, size_nm]
    ]
    return {"shape": "polygon", "polygon_nm": {"points": points}}

def generate_apertures(substrate_id: str, num_apertures: int) -> List[Aperture]:
    """Generates a simple list of apertures."""
    return [
        Aperture(uid=f"{substrate_id}_A{i}", index=i, status="available")
        for i in range(num_apertures)
    ]

async def generate_temdb_data(
    client: AsyncTEMdbClient,
    specimen_prefix: str = "GENSPEC",
    block_prefix: str = "GENBLOCK",
    session_prefix: str = "GENCUT",
    substrate_prefix: str = "GENMEDIA",
    media_type: Literal['tape', 'wafer', 'grid'] = 'tape',
    num_substrates: int = 1,
    num_apertures_per_substrate: int = 100,
    total_sections_to_create: Optional[int] = None,
    create_rois: bool = True,
    create_tasks: bool = True,
    task_batch_size: int = 500
):
    """
    Generates test data in TEMdb based on specified parameters.

    Args:
        client: An initialized AsyncTEMdbClient instance.
        specimen_prefix: Prefix for the generated specimen ID.
        block_prefix: Prefix for the generated block ID.
        session_prefix: Prefix for the generated cutting session ID.
        substrate_prefix: Prefix for the generated substrate media IDs.
        media_type: Type of substrate ('tape', 'wafer', 'grid').
        num_substrates: Number of substrate instances to create.
        num_apertures_per_substrate: Number of apertures/slots per substrate.
                                    For 'tape', this acts as max sections per tape.
        total_sections_to_create: Total number of sections desired. If None, creates
                                 one section per available aperture across all substrates.
        create_rois: Whether to create ROIs for the sections.
        create_tasks: Whether to create Acquisition Tasks for the ROIs.
        task_batch_size: Number of tasks to create in each batch request.
    """
    print("\n--- Starting Data Generation ---")
    print(f"Parameters: media_type={media_type}, num_substrates={num_substrates}, "
          f"apertures_per_substrate={num_apertures_per_substrate}, "
          f"total_sections={total_sections_to_create or 'all available'}, "
          f"create_rois={create_rois}, create_tasks={create_tasks}")

    specimen_id = f"{specimen_prefix}001"
    print(f"Creating Specimen: {specimen_id}...")
    specimen_data = SpecimenCreate(
        specimen_id=specimen_id,
        description=f"Generated specimen ({media_type})"
    )
    specimen = await client.specimen.create(specimen_data)
    print(f"  -> Created Specimen: {specimen.specimen_id}")

    block_id = f"{block_prefix}001"
    print(f"Creating Block: {block_id}...")
    block_data = BlockCreate(
        block_id=block_id,
        specimen_id=specimen.specimen_id,
        microCT_info={"source": "generated_data"}
    )
    block = await client.block.create(block_data)
    print(f"  -> Created Block: {block.block_id}")

    print(f"Creating {num_substrates} Substrate(s) of type '{media_type}'...")
    created_substrates = []
    all_substrate_apertures = [] 
    for i in range(num_substrates):
        substrate_media_id = f"{substrate_prefix}{i+1:03d}"
        substrate_uid = substrate_media_id 
        print(f"  Creating Substrate: {substrate_media_id}...")

        apertures = generate_apertures(substrate_media_id, num_apertures_per_substrate)
        all_substrate_apertures.append(apertures)

        substrate_data = SubstrateCreate(
            media_id=substrate_media_id,
            media_type=media_type,
            apertures=apertures,
            status="new",
            uid=substrate_uid
        )
        substrate = await client.substrate.create(substrate_data)
        created_substrates.append(substrate)
        print(f"    -> Created Substrate: {substrate.media_id}")

    total_available_apertures = num_substrates * num_apertures_per_substrate
    sections_to_make = total_sections_to_create if total_sections_to_create is not None else total_available_apertures
    sections_to_make = min(sections_to_make, total_available_apertures) 

    if sections_to_make <= 0:
        print("No sections to create based on parameters.")
        print("--- Data Generation Finished ---")
        return

    session_id = f"{session_prefix}001"
    print(f"Creating Cutting Session: {session_id}...")
    session_data = CuttingSessionCreate(
        cutting_session_id=session_id,
        block_id=block.block_id,
        start_time=datetime.datetime.now(datetime.timezone.utc),
        sectioning_device="GeneratedDevice",
        media_type=media_type,
        specimen_id=specimen.specimen_id
    )
    session = await client.cutting_session.create(session_data)
    print(f"  -> Created Cutting Session: {session.cutting_session_id}")

    print(f"Creating {sections_to_make} Section(s)...")
    all_sections = []
    assigned_aperture_count = 0
    for i in range(sections_to_make):
        section_num = i + 1

        substrate_idx = assigned_aperture_count // num_apertures_per_substrate
        aperture_idx_on_substrate = assigned_aperture_count % num_apertures_per_substrate

        if substrate_idx >= len(created_substrates):
            print(f"  WARNING: Ran out of substrates before creating all requested sections. Stopping at {i} sections.")
            break

        current_substrate = created_substrates[substrate_idx]
        current_aperture = all_substrate_apertures[substrate_idx][aperture_idx_on_substrate]

        section_data = SectionCreate(
            cutting_session_id=session.cutting_session_id,
            media_id=current_substrate.media_id,
            section_number=section_num,
            timestamp=datetime.datetime.now(datetime.timezone.utc),
            aperture_index=current_aperture.index,
            aperture_uid=current_aperture.uid
        )
        try:
            created_section = await client.section.create(section_data)
            all_sections.append(created_section)
            assigned_aperture_count += 1
            if section_num % 100 == 0:
                 print(f"  Created section {section_num}/{sections_to_make}")
        except Exception as e:
            print(f"  ERROR: Failed to create section {section_num}: {e}")
            raise e 
    print(f"  -> Finished creating {len(all_sections)} sections.")

    if not create_rois or not all_sections:
        print("Skipping ROI creation.")
    else:
        print(f"Creating {len(all_sections)} ROI(s)...")
        all_rois = []
        roi_params = create_square_roi_params()
        for i, section in enumerate(all_sections):
            # TODO: Using section_number as roi_id might cause collisions if run multiple times
            # or across different sessions. We need a more robust unique ID generation.
            roi_id = f"{specimen_id}__{substrate_media_id}_{substrate_uid}_{section.section_number}" #TEMP
            roi_data = ROICreate(
                roi_id=roi_id,
                section_id=section.section_id,
                specimen_id=section.specimen_id,
                block_id=section.block_id,
                section_number=section.section_number,
                roi_parameters=roi_params
            )
            try:
                created_roi = await client.roi.create(roi_data)
                all_rois.append(created_roi)
                if (i + 1) % 100 == 0: 
                    print(f"  Created ROI {i+1}/{len(all_sections)}")
            except Exception as e:
                print(f"  ERROR: Failed to create ROI for section {section.section_id}: {e}")
                raise e
        print(f"  -> Finished creating {len(all_rois)} ROIs.")

        
        if not create_tasks or not all_rois:
            print("Skipping Acquisition Task creation.")
        else:
            print(f"Creating {len(all_rois)} Acquisition Task(s) via batch...")
            tasks_to_create: List[AcquisitionTaskCreate] = []
            for roi in all_rois:
                task_id = f"TASK_{roi.specimen_id}_{roi.block_id}_{roi.roi_id}"
                task_data = AcquisitionTaskCreate(
                    task_id=task_id,
                    specimen_id=roi.specimen_id,
                    block_id=roi.block_id,
                    roi_id=roi.roi_id,
                    tags=[f"generated_{media_type}"]
                )
                tasks_to_create.append(task_data)

            if tasks_to_create:
                try:
                    created_tasks_count = 0
                    for j in range(0, len(tasks_to_create), task_batch_size):
                        chunk = tasks_to_create[j:j + task_batch_size]
                        print(f"  Submitting task batch {j//task_batch_size + 1} ({len(chunk)} tasks)...")
                        batch_result = await client.acquisition_task.create_batch(chunk)
                        created_tasks_count += len(batch_result)
                        print(f"    -> Batch created {len(batch_result)} tasks.")
                    print(f"  -> Finished creating {created_tasks_count} acquisition tasks via batch.")
                except Exception as e:
                     print(f"  ERROR: Failed to create task batch: {e}")
            else:
                 print("No ROIs available to create tasks for.")

    print("--- Data Generation Finished ---")


async def main():
    """Main function to run different generation scenarios."""
    print("Initializing TEMdb client for generation...")
    client: AsyncTEMdbClient = create_client(
        base_url=API_BASE_URL,
        api_key=API_KEY,
        async_mode=True,
        debug=False
    )

    async with client:
        await generate_temdb_data(
            client=client,
            specimen_prefix="TAPE_SPEC",
            media_type='tape',
            num_substrates=2,
            num_apertures_per_substrate=25,
            total_sections_to_create=50
        )
        await generate_temdb_data(
            client=client,
            specimen_prefix="WAFER_SPEC",
            media_type='wafer',
            num_substrates=1,
            num_apertures_per_substrate=100,
            total_sections_to_create=100 
        )

        await generate_temdb_data(
            client=client,
            specimen_prefix="GRID_SPEC",
            media_type='grid',
            num_substrates=1,
            num_apertures_per_substrate=1,
            total_sections_to_create=1
        )

        await generate_temdb_data(
            client=client,
            specimen_prefix="MULTIGRID_SPEC",
            media_type='grid',
            num_substrates=10, 
            num_apertures_per_substrate=1,
            total_sections_to_create=10
        )

        await generate_temdb_data(
            client=client,
            specimen_prefix="NOTASK_SPEC",
            media_type='tape',
            num_substrates=1,
            num_apertures_per_substrate=20,
            total_sections_to_create=20,
            create_tasks=False 
        )


if __name__ == "__main__":
    print("Running Generic TEMdb Data Generator Script...")
    logging.basicConfig(level=logging.INFO)
    
    try:
        loop = asyncio.get_running_loop()
        print("Detected running event loop. Scheduling coroutine.")
        if loop.is_running():
             print("WARNING: Cannot reliably run in already running loop from __main__.")
             print("         Consider running 'await main()' directly in the cell/environment.")
        else:
             loop.run_until_complete(main())
    except RuntimeError:
        print("No running event loop detected. Using asyncio.run().")
        asyncio.run(main())

    print("Script completed.")



In [ ]:
await main()

In [ ]:
print(client.get_example_usage())

### Specimens

#### Create

In [ ]:
await client.specimen.create({
  "specimen_id": "example_specimen_1",
  "description": "An example specimen",
  "created_at": datetime.now().isoformat(),
  "updated_at": datetime.now().isoformat(),
})

#### List

In [ ]:
await client.specimen.list()

#### Update

In [ ]:
await client.specimen.update(
  "example_specimen_1",
  {"description": "A new description!"},
)

#### Get

In [ ]:
await client.specimen.get("example_specimen_1")

### Block

#### Create

In [ ]:
await client.block.create({
  "block_id": "example_block_1",
  "specimen_id": "example_specimen_1",
  "microCT_info": {},
})

#### List

In [ ]:
await client.block.list("example_specimen_1")

#### Update

In [ ]:
await client.block.update(
  "example_specimen_1",
  "example_block_1",
  {"microCT_info": {"example_key": "example data"}},
)

#### Get

In [ ]:
await client.block.get("example_specimen_1", "example_block_1")

### Cutting Session

#### Create

In [ ]:
await client.cutting_session.create({
  "cutting_session_id": "example_cutting_session_1",
  "start_time": datetime.now().isoformat(),
  "operator": "me",
  "sectioning_device": "cookie",
  "media_type": "tape",
  "specimen_id": "example_specimen_1",
  "block_id": "example_block_1",
})

#### List

In [ ]:
await client.cutting_session.list("example_specimen_1", "example_block_1")

#### Update

In [ ]:
await client.cutting_session.update(
  "example_cutting_session_1",
  {"end_time": datetime.now().isoformat()},
)

#### Get

In [ ]:
await client.cutting_session.get("example_specimen_1", "example_block_1", "example_cutting_session_1")

### Section

#### Create

In [ ]:
await client.section.create({
  "section_id": "example_section_1",
  "section_number": 1,
  "optical_image": {},
  "section_metrics": {
    "sectioning_metadata": {},
  },
  "media_type": "tape",
  "media_id": "example_media",
  "relative_position": 10,
  "barcode": "example_barcode",
  "cutting_session_id": "example_cutting_session_1",
})
  

#### List

In [ ]:
await client.section.list("example_cutting_session_1")

#### Update

In [ ]:
await client.section.update(
  "example_cutting_session_1",
  "example_section_1",
  {"section_metrics": {"tissue_confidence_score": 0.5}},
)

#### Get

In [ ]:
await client.section.get("example_cutting_session_1", "example_section_1")

### ROI

#### Create

In [ ]:
await client.roi.create({
  "roi_id": 1,
  "specimen_id": "example_specimen_1",
  "block_id": "example_block_1",
  "section_number": 1,
  "section_id": "example_section_1",
  "aperture_width_height": [1, 2],
  "aperture_centroid": [10, 11],
  "aperture_bounding_box": [1, 2, 3, 4],
  "optical_nm_per_pixel": 250,
  "barcode": "example_barcode",
})

#### List

In [ ]:
await client.roi.list("example_section_1")

#### Update

In [ ]:
await client.roi.update(1, {"updated_at": datetime.now().isoformat()})

#### Get

In [ ]:
await client.roi.get(1)

### Acqusition Task Session

#### Create

In [ ]:
await client.acquisition_task.create({
  "session_id": "example_imaging_session_1",
  "specimen_id": "example_specimen_1",
  "block_id": "example_block_1",
  "media_type": "tape",
  "media_id": "example media ID",
  "start_time": datetime.now().isoformat(),
  "status": "In Progress",
  "rois": [1],
})

#### List

In [ ]:
await client.acquisition_task.list("example_specimen_1")

#### Update

In [ ]:
await client.acquisition_task.update(
  "example_imaging_session_1",
  {
    "status": "Completed",
    "end_time": datetime.now().isoformat(),
  }
)

#### Get

In [ ]:
await client.acquisition_task.get("example_imaging_session_1")

### Acquisition

#### Create

In [ ]:
await client.acquisition.create({
  "specimen_id": "example_specimen_1",
  "montage_id": "example_montage_1",
  "acquisition_id": "example_acquisition_1",
  "roi_id": 1,
  "imaging_session_id": "example_imaging_session_1",
  "hardware_settings": {
    "scope_id": "T1",
    "camera_model": "MX1276",
    "bit_depth": 10,
    "camera_serial": "123abc",
    "media_type": "tape",
  },
  "calibration_settings": {
      "pixel_size": 0.5,
      "stig_angle": 0.5,
  },
  "acquisition_settings": {
    "magnification": 2000,
    "spot_size": 4,
    "exposure_time": 125,
    "tile_size": [5000, 5000],
    "tile_overlap": 0.05,
  },
  "tilt_angle": 0,
  "lens_correction": False,
  "start_time": datetime.now().isoformat(),
})

#### List

In [ ]:
await client.acquisition.list()

#### Update

In [ ]:
await client.acquisition.update(
  "example_acquisition_1",
  {
    "end_time": datetime.now().isoformat(),
  }
)

#### Get

In [ ]:
await client.acquisition.get("example_acquisition_1")

### Tile

#### Create

In [ ]:
await client.acquisition.add_tile(
  "example_acquisition_2",
  {
    "tile_id": "example_tile_2",
    "raster_index": 1,
    "stage_position": {"x": 120, "y": 150},
    "raster_position": {"x": 10, "y": 5},
    "focus_score": 1,
    "min_value": 0,
    "max_value": 128,
    "mean_value": 55,
    "std_value": 24,
    "image_path": "example s3 path",
    "matcher": [],
  }
)
  

#### Count

In [ ]:
await client.acquisition.get_tile_count("example_acquisition_2")

#### Get

In [ ]:
await client.acquisition.get_tile("example_acquisition_2", "example_tile_2")